In [ ]:
INPUT_FOLDER = '/Users/bober/Desktop/study/naukma/prog/data'

INPUT_FILE_MERGED = 'merged_dataset_v1.csv'
INPUT_FILE_ISW = 'isw_all_days_parsed_v1.csv'
INPUT_FOLDER = '/Users/bober/Desktop/study/naukma/prog/data'

MODEL_FOLDER = "models"


tfidf_transformer_model = "tfidf_transformer"
count_vectorizer_model = "count_vectorizer"

tfidf_transformer_version = "v1"
count_vectorizer_version = "v1"

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

In [ ]:
import pandas as pd
import datetime
import pickle
import scipy
import numpy as np

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter
from num2words import num2words
from nltk.corpus import stopwords
import nltk

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
df_raw_data = pd.read_csv(f'{INPUT_FOLDER}/{INPUT_FILE_MERGED}',sep = ';')

In [ ]:
df_raw_data

In [ ]:
#df_raw_data[df_raw_data['day_datetime']=='2022-04-28']

In [ ]:
is_alarm = ~((df_raw_data['event_end_hour'].isna()) & (df_raw_data['event_start_hour'].isna()))

In [ ]:
df_raw_data['is_alarm']= is_alarm
df_raw_data

In [ ]:
df_raw_data.describe()


In [ ]:
df_isw_raw = pd.read_csv(f'{INPUT_FOLDER}/{INPUT_FILE_ISW}',sep = ';')
df_isw_raw

### dropping the row and selecting the necessary columns/


In [ ]:
#pd.to_datetime(df_isw_raw['date'])+datetime.timedelta(days=1)
df_isw_raw['report_date']=df_isw_raw['date']
df_isw_raw['date_tmrw']= (pd.to_datetime(df_isw_raw['date'])+datetime.timedelta(days=1)).dt.strftime('%Y-%m-%d')


In [ ]:
df_isw_upd = df_isw_raw.drop( labels = 0, axis=0,inplace=False)[["report_date", "date_tmrw", "keywords", "main_html_v7", "report_text_lemm"]].copy().add_prefix('isw_')


In [ ]:
df_isw_upd

In [ ]:
df_all = df_raw_data.merge(df_isw_upd, how = 'left', left_on = 'day_datetime', right_on = 'isw_date_tmrw')

In [ ]:
df_all.sample(5)

In [ ]:
df_all

In [ ]:
to_delete = ['city_resolvedAddress',
'day_datetime',
"hour_datetime",
"hour_datetimeEpoch",
"city",
"region",
"center_city_ua",
"center_city_en",
"event_region_title",
"event_region_city",
"event_all_region",
"event_clean_end",
"event_intersection_alarm_id",
"event_start_hour",
"event_end_hour",
"event_start_hour_datetimeEpoch",
"event_end_hour_datetimeEpoch",
"event_hour_level_event_time",
"event_hour_level_event_datetimeEpoch",
"isw_report_date",
'isw_date_tmrw',
'isw_main_html_v7',
'isw_keywords',
'isw_report_text_lemm'
]

In [ ]:
df_weather_matrix = df_all.drop(to_delete, axis = 1)
df_weather_matrix

In [ ]:
additional_to_delete = [
"day_sunset",
"hour_preciptype",
"hour_conditions",
"region_alt",
"event_start",
"event_end",
"hour_solarenergy",
'event_region_id',
'event_duration',
'event_duration_in_mins_int',
'event_amount_of_alarms_for_one_day',
'event_intersection_alarm_region_counts',
'event_last_24_hour_in_region',
'event_start_date',
'event_end_date',
'day_sunrise'
]

In [ ]:
df_weather_matrix_v1 = df_weather_matrix.drop(additional_to_delete, axis = 1)


In [ ]:
df_weather_matrix_v2 = df_weather_matrix_v1.drop(['is_alarm'],axis=1 ).fillna(method="ffill")

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_weather_matrix_v2.sample(5)

In [ ]:
df_weather_matrix_v2_csr = scipy.sparse.csr_matrix(df_weather_matrix_v2.values)


### TD-IDF

In [ ]:
tf_idf = pickle.load(open ("/Users/bober/Desktop/study/naukma/prog/td/tfidf_transformer_v1.pkl", "rb"))
cv = pickle.load(open ("/Users/bober/Desktop/study/naukma/prog/td/count_vectorizer_v1.pkl", "rb"))

In [ ]:
df_isw_upd['isw_report_text_lemm']

In [ ]:
type(tf_idf)

In [ ]:
cv_vector = cv.transform(df_all['isw_report_text_lemm'].values.astype('U'))


In [ ]:
tf_idf_vector = tf_idf.transform(cv_vector)

In [ ]:
tf_idf_vector.shape

In [ ]:
df_all_data_csr = scipy.sparse.hstack((df_weather_matrix_v2_csr, tf_idf_vector), format='csr')

In [ ]:
df_all_data_csr

# TRAIN

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_decomposition import PLSCanonical

In [ ]:
#ts_cv = TimeSeriesSplit(
 #   n_splits=5,
 #   gap=48,
 #   max_train_size=None,
 #   test_size=40000,
#)
#array = ts_cv.split(df_all_data_csr,is_alarm)


In [ ]:
def get_param(matrix):
    TP = matrix[0,0]
    FP = matrix[0,1]
    FN = matrix[1,0]
    TN = matrix[1,1]
    recall = TP/(TP+FN)
    precision = TP/(TP+FP)
    accuracy = (TP+TN)/(TP+FP+TN+FN)
    result = f'Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}'
    return result

In [ ]:
RANDOM_STATE = 1
X_train, X_test, y_train, y_test = train_test_split(df_all_data_csr, is_alarm,
test_size=0.20,
random_state = RANDOM_STATE,
shuffle=True)

# 1 too long

In [ ]:
model_1 = svm.SVC()
model_1.fit(X_train, y_train)

In [ ]:
predicted1 = model_1.predict(X_test)
np.mean(predicted1 == y_test)

In [ ]:
confusion_matrix_1 = confusion_matrix(y_test, predicted1)
confusion_matrix_1

In [ ]:
get_param(confusion_matrix_1)

# 2 

In [ ]:
model_2 =  LogisticRegression(penalty="l1", tol=0.01, solver="saga")
model_2.fit(X_train, y_train)

In [ ]:
predicted2 = model_2.predict(X_test)
np.mean(predicted2 == y_test)

In [ ]:
confusion_matrix_2 = confusion_matrix(y_test, predicted2)
confusion_matrix_2

In [ ]:
get_param(confusion_matrix_2)

In [ ]:
import matplotlib.pyplot as plt
coef = model_2.coef_.ravel()
coef
top_features = np.argsort(np.abs(coef))[-20:]
top_weights = coef[top_features]

plt.figure(figsize=(10, 8))
plt.barh(range(len(top_features)), top_weights, align='center')
plt.yticks(range(len(top_features)), [f'Feature {i}' for i in top_features])
plt.xlabel('Feature Weight')
plt.title('Top 20 Features with their Weights')
plt.show()

# 3 done

In [ ]:
model_3 = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
model_3.fit(X_train, y_train)

In [ ]:
predicted3 = model_3.predict(X_test)
np.mean(predicted3 == y_test)

In [ ]:
confusion_matrix_3 = confusion_matrix(y_test, predicted3)
confusion_matrix_3

In [ ]:
get_param(confusion_matrix_3)

# 4 done

In [ ]:
model_4 = RandomForestClassifier()
model_4.fit(X_train, y_train)

In [ ]:
predicted4 = model_4.predict(X_test)
np.mean(predicted4 == y_test)

In [ ]:
confusion_matrix_4 = confusion_matrix(y_test, predicted4)
confusion_matrix_4

In [ ]:
get_param(confusion_matrix_4)

# 5 done

In [ ]:
model_5 = tree.DecisionTreeClassifier()
model_5.fit(X_train, y_train) 

In [ ]:
predicted5 = model_5.predict(X_test)
np.mean(predicted5 == y_test)

In [ ]:
confusion_matrix_5 = confusion_matrix(y_test, predicted5)
confusion_matrix_5

In [ ]:
get_param(confusion_matrix_5)

# 6

In [ ]:
model_6 = KNeighborsClassifier()
model_6.fit(X_train, y_train)


In [ ]:
predicted6 = model_6.predict(X_test)
np.mean(predicted6 == y_test)

In [ ]:
confusion_matrix_6 = confusion_matrix(y_test, predicted6)
confusion_matrix_6

In [ ]:
get_param(confusion_matrix_6)

## Saving model

In [ ]:
INPUT_FOLDER_2 = '/Users/bober/Desktop/study/naukma/prog/models'

In [ ]:
name = 'random_forest'
version = 'v1'
name_model = f'6__{name}__{version}'
model = model_4

with open(f'{INPUT_FOLDER_2}/{name_model}.pkl','wb') as handle:
    pickle.dump(model, handle)

## Calculating a confusion matrix

In [ ]:
confusion_matrix(y_true, y_pred)